Copyright Preferred Computational Chemistry, Inc. as contributors to Matlantis contrib project

# nglviewerでのTrajectory 表示 - Util Example

デフォルトの`nglviewer` では、Trajectoryを表示する際、一番はじめのAtoms に対してBond, Cell のみを設定して、Frameの移動時には座標値のみが更新されます。そのため、

 - Bondの変化 (切れたりつながったり)
 - Cell sizeの変化
 - 原子数の増減　(Ex. LAMMPSでのEvapolateやDepositに相当する操作)
 - 元素種の変化

などがあるようなTrajectoryは正しく表示できません。

以下の例のように、`observe_replace_structure` 関数を用いることでこれらのケースも表示することができます。(ただし、毎回Bond 表示更新などの計算が走るため、少し動作は重くなります。)

In [ ]:
!pip install ase

# # 初回使用時のみ、ライブラリのインストールをお願いします。

In [1]:
from nglview import ASEStructure
from nglview.widget import NGLWidget
from nglview.adaptor import ASETrajectory


def get_struct(atoms, ext="pdb"):
    struct = ASEStructure(atoms, ext=ext).get_structure_string()
    for c in range(len(atoms)):
        struct = struct.replace("MOL     1", "M0    " + str(c).zfill(3), 1)
    struct = [dict(data=struct, ext=ext)]
    return struct


def observe_replace_structure(v: NGLWidget):
    traj = v._trajlist[0]
    if not isinstance(traj, ASETrajectory):
        print(
            "[WARNING] ASETrajectory is expected, other type is not tested..."
        )
    struct_list = [None for _ in range(traj.n_frames)]

    def _on_frame_changed_replace_structure(change):
        """set and send coordinates at current frame"""
        struct = struct_list[v.frame]
        if struct is None:
            atoms = traj.trajectory[v.frame]
            struct = get_struct(atoms)
            struct_list[v.frame] = struct  # Cache
        v._remote_call("replaceStructure", target="Widget", args=struct)

    # Remove original `_on_frame_changed` callback which only updates coordinates
    v.unobserve(NGLWidget._on_frame_changed)
    # Add `_on_frame_changed_replace_structure` callback which updates coordinates, bonds, unitcells etc.
    v.observe(_on_frame_changed_replace_structure, names=["frame"])


In [2]:
import nglview
nglview.__version__

'3.0.1'

## 例１：Bond有無やCell sizeが変わるTrajectory表示

In [3]:
import ase
from ase.build import bulk, molecule, surface, add_adsorbate
from ase.io import read
from ase.visualize import view

# A. Jain*, S.P. Ong*, G. Hautier, W. Chen, W.D. Richards, S. Dacek, S. Cholia, D. Gunter, D. Skinner, G. Ceder, K.A. Persson (*=equal contributions)
# The Materials Project: A materials genome approach to accelerating materials innovation
# APL Materials, 2013, 1(1), 011002.
atoms = ase.io.read("input/SiO2_mp-6945_computed.cif") * (2, 2, 2)

atoms_list = []
for i in range(20):
    atoms_tmp = atoms.copy()
    scaled_pos = atoms_tmp.get_scaled_positions()
    # atoms_tmp.cell = atoms_tmp.cell * (1 + 0.1 * i)
    atoms_tmp.cell[2] = atoms_tmp.cell[2] * (1 + 0.05 * i)
    atoms_tmp.cell[1] = atoms_tmp.cell[1] * (1 + 0.05 * i)
    atoms_tmp.set_scaled_positions(scaled_pos)
    # print(atoms_tmp.positions)
    atoms_list.append(atoms_tmp)

v = view(atoms_list, viewer='ngl')
# v.view.add_representation("ball+stick")
v.view.add_ball_and_stick()

observe_replace_structure(v.view)

v

## 例２：元素種の変化や原子数が増減するTrajectory表示


`observe_replace_structure(v.view)` なしでは、原子数が変わる Trajectoryの表示ができませんが、`observe_replace_structure`ありだと表示できることがわかります。

In [4]:
from ase.collections import g2
print("Available molecule:", len(g2.names), g2.names)

Available molecule: 162 ['PH3', 'P2', 'CH3CHO', 'H2COH', 'CS', 'OCHCHO', 'C3H9C', 'CH3COF', 'CH3CH2OCH3', 'HCOOH', 'HCCl3', 'HOCl', 'H2', 'SH2', 'C2H2', 'C4H4NH', 'CH3SCH3', 'SiH2_s3B1d', 'CH3SH', 'CH3CO', 'CO', 'ClF3', 'SiH4', 'C2H6CHOH', 'CH2NHCH2', 'isobutene', 'HCO', 'bicyclobutane', 'LiF', 'Si', 'C2H6', 'CN', 'ClNO', 'S', 'SiF4', 'H3CNH2', 'methylenecyclopropane', 'CH3CH2OH', 'F', 'NaCl', 'CH3Cl', 'CH3SiH3', 'AlF3', 'C2H3', 'ClF', 'PF3', 'PH2', 'CH3CN', 'cyclobutene', 'CH3ONO', 'SiH3', 'C3H6_D3h', 'CO2', 'NO', 'trans-butane', 'H2CCHCl', 'LiH', 'NH2', 'CH', 'CH2OCH2', 'C6H6', 'CH3CONH2', 'cyclobutane', 'H2CCHCN', 'butadiene', 'C', 'H2CO', 'CH3COOH', 'HCF3', 'CH3S', 'CS2', 'SiH2_s1A1d', 'C4H4S', 'N2H4', 'OH', 'CH3OCH3', 'C5H5N', 'H2O', 'HCl', 'CH2_s1A1d', 'CH3CH2SH', 'CH3NO2', 'Cl', 'Be', 'BCl3', 'C4H4O', 'Al', 'CH3O', 'CH3OH', 'C3H7Cl', 'isobutane', 'Na', 'CCl4', 'CH3CH2O', 'H2CCHF', 'C3H7', 'CH3', 'O3', 'P', 'C2H4', 'NCCN', 'S2', 'AlCl3', 'SiCl4', 'SiO', 'C3H4_D2d', 'H', 'COF2', '

In [5]:
from ase.build import molecule

atoms0 = molecule("CH3CHO")
atoms1 = molecule("C60")

In [6]:
v = view([atoms0, atoms1], viewer='ngl')
v.view.add_representation("ball+stick")
observe_replace_structure(v.view)

display(v)